In [ ]:
#reconstruct phase
import numpy as np
import cmath
import math
phase_half_pi_calibrated=math.pi/2+math.pi*0.1
size=25

def getMinMax(arr):
    arr.sort()
    minmax = {"min": arr[0], "max": arr[-1]}
    return minmax


In [ ]:
#phase noise on arm2
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
amplitude=100*np.pi
time_range=10
period_sine=1
num_period_sine=int(time_range/period_sine)
samples_per_period=3000
time = np.linspace(0, num_period_sine*period_sine, num_period_sine*samples_per_period, endpoint=False)
#phase_noise=amplitude*np.sin(2*np.pi * time)
phase_noise=amplitude*time
plt.figure(figsize=(5,3))
plt.plot(time, phase_noise/np.pi)
plt.title("phase noise on arm2")
plt.ylabel("rads in pi",fontsize=10)
plt.xlabel("time in second",fontsize=10)
plt.tick_params(axis='both', which='major', labelsize=10)

print("number of dots in sine wave",num_period_sine*samples_per_period)


#plt.plot(t, )
#plt.ylim(-2, 2)

In [ ]:


r=1/math.sqrt(2); #cross_coupling_coefficient for the directional coupler (0.707 is 1/sqrt(2)) 
t=math.sqrt(1-r**2); #optical_field_transmission
transfer_directional_coupler=np.matrix([[t,r*1j],[r*1j,t]]) #transfer matrix for directional coupler 3dB coupler
matrixs2=np.dot([1,0],transfer_directional_coupler); #incident on the first arm; the element is 0.5**0.5

l=0  #buffer for count
phase_measure_each = np.empty(len(phase_noise)) 
phase_measure = np.empty(len(phase_noise)) 
phase_error = np.empty(len(phase_noise)) 

#assume we correct phase after each measurement
for i in range(0, len(phase_noise)):
    Ey_arm1_Ey_arm2=[matrixs2[0,0],matrixs2[0,1]]; #arm2 arm1 ER is infinity
    for k in range(0,2):
        phase_angle_one=0; 
        if k==0:
            phase_angle_two=-math.pi/2;
        elif k==1:
            phase_angle_two=0;
        phase_shifter=np.matrix([[np.exp(1j*(phase_angle_one)),0],[0,np.exp(1j*(phase_angle_two+phase_noise[i]))]]);
        matrixs3=Ey_arm1_Ey_arm2@phase_shifter@transfer_directional_coupler;
        E1=abs(matrixs3[0,0]);
        E2=abs(matrixs3[0,1]);
        I1_and_I2=[E1**2,E2**2];
        if k==0:
            I_diff=I1_and_I2[1]-I1_and_I2[0];
        elif k==1:
            Q_diff=I1_and_I2[1]-I1_and_I2[0];
    
    #fix bug of this part:
    if Q_diff==0:
        if I_diff>0:
            phase_measure_each[i]=np.pi/2
        if I_diff<0:
            phase_measure_each[i]=-np.pi/2
    # tricky

    else:    
        phase_measure_each[i]=np.arctan(I_diff/Q_diff)
    if i>=1:
        if phase_measure_each[i]-phase_measure_each[i-1]>math.pi*0.5:
            l=l-1
        if phase_measure_each[i]-phase_measure_each[i-1]<-math.pi*0.5:
            l=l+1
    phase_measure[i]=l*np.pi+phase_measure_each[i]

time = np.linspace(0, num_period_sine*period_sine, num_period_sine*samples_per_period, endpoint=False)

plt.figure(figsize=(5,3))
plt.plot(time, phase_measure/np.pi)
plt.title("phase measured")
plt.ylabel("rads in pi",fontsize=10)
plt.xlabel("time in second",fontsize=10)
plt.tick_params(axis='both', which='major', labelsize=10)

minmax = getMinMax(phase_measure_each) 
print("Minimum element is", minmax["min"])
print("Maximum element is", minmax["max"])
print("peak-peak", minmax["max"]-minmax["min"])

#print("  phase_measure", phase_measure/np.pi)
#print("  ")
#print("  phase_measure_each", phase_measure_each/np.pi)



In [ ]:
#create ER array
from scipy import signal

frequency_square=1
period_square=1/frequency_square
num_period_square=int(time_range/period_square)
samples_per_period_square=int(samples_per_period/frequency_square)
time2 = np.linspace(0, num_period_square*period_square, num_period_square*samples_per_period_square, endpoint=False)
ER_array_1=15*signal.square((2 * np.pi)* frequency_square*(time2),duty=0.5)-30
#ER_array_2=30*signal.square((2 * np.pi)* frequency_square*(time2),duty=0.5)-35
#ER_array_2=15*np.sin((1 * np.pi)* frequency_square* (time2+0.2))-30
ER_array_2=15*signal.square((2 * np.pi)* frequency_square*(time2+period_square/2),duty=0.5)-30
#plt.figure(figsize=(30,3))
fig, axs = plt.subplots(1, 2)
fig.set_figheight(5)
fig.set_figwidth(25)

axs[0].plot(time2, ER_array_1)
axs[0].set_title("ER1 array on arm1")
axs[0].set(xlabel="time in second", ylabel='intensity in dB')


axs[1].plot(time2, ER_array_2)
axs[1].set_title("ER array on arm2")
axs[1].set(xlabel="time in second", ylabel='intensity in dB')

print("number of dots in square wave",num_period_square*samples_per_period_square)


In [ ]:
size=20
#phase error when consider ER
import numpy as np
import cmath
import math

phase_measure_each_2 = np.empty(len(phase_noise)) 
phase_measure_2 = np.empty(len(phase_noise))
phase_error_2= np.empty(len(phase_noise))
half_pi_error=0 #0.2*math.pi/2
l=0

for i in range(0, len(phase_noise)):
    Ey_arm1_ER=(0.5*(1-10**(ER_array_1[i]/10)))**0.5; #the power
    Ey_arm2_ER=(0.5*(1-10**(ER_array_2[i]/10)))**0.5; #the power
    Ey_arm1_Ey_arm2=[Ey_arm1_ER,Ey_arm2_ER]
    for k in range(0,2):
        phase_angle_one=0; 
        if k==0:
            phase_angle_two=-math.pi/2;
        elif k==1:
            phase_angle_two=half_pi_error;
        phase_shifter=np.matrix([[np.exp(1j*(phase_angle_one)),0],[0,np.exp(1j*(phase_angle_two+phase_noise[i]))]]);
        matrixs3=Ey_arm1_Ey_arm2@phase_shifter@transfer_directional_coupler;
        E1=abs(matrixs3[0,0]);
        E2=abs(matrixs3[0,1]);
        I1_and_I2=[E1**2,E2**2];
        if k==0:
            I_diff=I1_and_I2[1]-I1_and_I2[0];
        elif k==1:
            Q_diff=I1_and_I2[1]-I1_and_I2[0];
    #fix bug of this part:
    if Q_diff==0:
        if I_diff>0:
            phase_measure_each_2[i]=np.pi/2
        if I_diff<0:
            phase_measure_each_2[i]=-np.pi/2
    # tricky
    else:    
        phase_measure_each_2[i]=np.arctan(I_diff/Q_diff)
    if i>=1:
        if phase_measure_each_2[i]-phase_measure_each_2[i-1]>math.pi*0.5:
            l=l-1
        if phase_measure_each_2[i]-phase_measure_each_2[i-1]<-math.pi*0.5:
            l=l+1
    phase_measure_2[i]=l*np.pi+phase_measure_each_2[i]
    #assume we correct phase after each measurement
    #consider ER
  
phase_error_2=phase_measure_2-phase_noise

plt.figure(figsize=(8,2))
plt.plot(time, phase_measure_2/np.pi)
plt.title("phase measure",fontsize=size)
plt.ylabel("rads in pi",fontsize=size)
plt.xlabel("time in second",fontsize=size)
plt.tick_params(axis='both', which='major', labelsize=size)

plt.figure(figsize=(16,4))
plt.plot(time, phase_error_2/np.pi)
plt.title("phase error ",fontsize=size)
plt.ylabel("rads in pi",fontsize=size)
plt.xlabel("time in second",fontsize=size)
#plt.ylim(-2.5-1e-13,-2.5+1e-13)
plt.tick_params(axis='both', which='major', labelsize=size)

print(len(phase_noise))
print("measuring frequency=",len(phase_noise)/time_range,"Hz")
print("50*wavelength=",50*2*math.pi,"rad")

print("  phase_measure 2", phase_measure_2/np.pi)
print("  ")
print("  phase_noise", phase_noise/np.pi)


In [ ]:
this is End of the file

In [ ]:
minmax = getMinMax(phase_error_accumulate) 
print("Minimum element is", minmax["min"])
print("Maximum element is", minmax["max"])

In [ ]:
#create ER array
import matplotlib as mpl
from cycler import cycler
mpl.rcParams['lines.linewidth'] = 2
#mpl.rcParams['lines.linestyle'] = '--'
data = np.random.randn(100)*5+30
plt.plot(data)
plt.ylabel("rads",fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=20)

In [ ]:

# Aleternative way how can complex no can initialize
# importing "cmath" for complex number operations
import cmath
n=3 
# Initializing complex number
z = 5+n*1j
# Print the parts of Complex No.
print("The real part of complex number is : ", end="")
print(z)
 
print("The imaginary part of complex number is : ", end="")
print(z.imag)

#### 